In [19]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [31]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=8,kernel_size=3,padding=0) #26
        self.bath1 = nn.BatchNorm2d(num_features=8)
        
        self.conv2 = nn.Conv2d(in_channels=8,out_channels=8,kernel_size=3,padding=0) #24 
        self.bath2 = nn.BatchNorm2d(num_features=8)
        
        self.conv3 = nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,padding=0) #22 
        self.bath3 = nn.BatchNorm2d(num_features=16)
        self.pool1 = nn.MaxPool2d(kernel_size=2,stride=2) #11
        
        
        self.conv4 = nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,padding=0) #9 
        self.bath4 = nn.BatchNorm2d(num_features=16)
        
        self.conv5 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding=0) #7
        self.bath5 = nn.BatchNorm2d(num_features=32)
        
        self.conv6 = nn.Conv2d(in_channels=32,out_channels=16,kernel_size=1,padding=0) #7
        self.bath6 = nn.BatchNorm2d(num_features=16)
        
        self.conv7 = nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,padding=0)  #5
        self.bath7 = nn.BatchNorm2d(num_features=16)
        
        self.conv8 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding=0) #3
        self.bath8 = nn.BatchNorm2d(num_features=32)
        
        self.conv9 = nn.Conv2d(in_channels=32,out_channels=10,kernel_size=3,padding=0) #1
        
        
    def forward(self, x):
        x = self.bath1(F.relu(self.conv1(x)))
        x = self.bath2(F.relu(self.conv2(x)))
        x = self.pool1(self.bath3(F.relu(self.conv3(x))))
        x = self.bath4(F.relu(self.conv4(x)))
        x = self.bath5(F.relu(self.conv5(x)))
        x = self.bath6(F.relu(self.conv6(x)))
        x = self.bath7(F.relu(self.conv7(x)))
        x = self.bath8(F.relu(self.conv8(x)))
        x = self.conv9(x)
        x = x.view(-1,10)
        return F.log_softmax(x)

In [32]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu") 
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
       BatchNorm2d-2            [-1, 8, 26, 26]              16
            Conv2d-3            [-1, 8, 24, 24]             584
       BatchNorm2d-4            [-1, 8, 24, 24]              16
            Conv2d-5           [-1, 16, 22, 22]           1,168
       BatchNorm2d-6           [-1, 16, 22, 22]              32
         MaxPool2d-7           [-1, 16, 11, 11]               0
            Conv2d-8             [-1, 16, 9, 9]           2,320
       BatchNorm2d-9             [-1, 16, 9, 9]              32
           Conv2d-10             [-1, 32, 7, 7]           4,640
      BatchNorm2d-11             [-1, 32, 7, 7]              64
           Conv2d-12             [-1, 16, 7, 7]             528
      BatchNorm2d-13             [-1, 16, 7, 7]              32
           Conv2d-14             [-1, 1

/home/jd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [33]:
torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [34]:
from tqdm import tqdm
l = []
acc = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    l.append(test_loss)
    acc.append(100. * correct / len(test_loader.dataset))

In [35]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0375, momentum=0.9)

for epoch in range(1, 21):
    print ('Epochs',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/235 [00:00<?, ?it/s]

Epochs 1


/home/jd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/235 [00:00<?, ?it/s]


Test set: Average loss: 0.0658, Accuracy: 9796/10000 (98%)

Epochs 2


  0%|          | 0/235 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0235, Accuracy: 9943/10000 (99%)

Epochs 19


  0%|          | 0/235 [00:00<?, ?it/s]


Test set: Average loss: 0.0253, Accuracy: 9937/10000 (99%)

Epochs 20


loss=0.004449824336916208 batch_id=234: 100%|██████████| 235/235 [00:09<00:00, 23.60it/s]  



Test set: Average loss: 0.0249, Accuracy: 9935/10000 (99%)

